
# Análisis y simulación de egresos

## Dependencies

In [11]:
import pandas as pd
from datetime import timedelta, date

## Import data

In [12]:
# Downloadable at https://repositoriodeis.minsal.cl/DatosAbiertos/EGRESOS/Egresos_Hospitalarios_2020.zip

cols = [
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "SEXO",
    "TIPO_EDAD",
    "EDAD_CANT",
    "ANO_EGRESO",
    "FECHA_EGRESO",
    "AREA_FUNCIONAL_EGRESO",
    "DIAS_ESTADA",
    "CONDICION_EGRESO",
    "DIAG1",
    "GLOSA_DIAG1",
    "DIAG2",
    "GLOSA_DIAG2",
    "INTERV_Q",
    "PROCED",
    "CODIGO_PROCED_PPAL",
]

with open("data/egresos_2020.csv", "r", encoding="latin-1") as f:
    df = pd.read_csv(
        f,
        engine="c",
        sep=";",
        encoding="iso-8859-1",
        parse_dates=["FECHA_EGRESO"],
        dayfirst=True,
        infer_datetime_format=True,
        error_bad_lines=False,
        usecols=cols
    )


## Filtering

In [13]:
# Filtrar para solo 2020 y el Hospital San José de Melipilla
patients = df[df["ESTABLECIMIENTO_SALUD"] == 110150].reset_index(drop=True)

## Prepare columns

In [14]:
patients.head()

,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,SEXO,EDAD_CANT,TIPO_EDAD,ANO_EGRESO,FECHA_EGRESO,AREA_FUNCIONAL_EGRESO,DIAS_ESTADA,CONDICION_EGRESO,DIAG1,GLOSA_DIAG1,DIAG2,GLOSA_DIAG2,INTERV_Q,PROCED,CODIGO_PROCED_PPAL
0,110150,Hospital San José (Melipilla),1,33,1,2020,2020-08-27,403,6,1,N481,BALANOPOSTITIS,NaN,NaN,2,2,NaN
1,110150,Hospital San José (Melipilla),2,81,1,2020,2020-01-22,403,11,1,I470,ARRITMIA POR REENTRADA VENTRICULAR,NaN,NaN,2,2,NaN
2,110150,Hospital San José (Melipilla),1,3,3,2020,2020-09-16,413,1,1,P599,"ICTERICIA NEONATAL, NO ESPECIFICADA",NaN,NaN,2,2,NaN
3,110150,Hospital San José (Melipilla),2,23,1,2020,2020-10-05,416,1,1,O600,TRABAJO DE PARTO PREMATURO SIN PARTO,NaN,NaN,2,2,NaN
4,110150,Hospital San José (Melipilla),2,55,1,2020,2020-04-17,403,12,1,I500,INSUFICIENCIA CARDIACA CONGESTIVA,NaN,NaN,2,2,NaN


In [15]:
patients["DIAS_ESTADA"] = pd.to_timedelta(patients["DIAS_ESTADA"], unit="days")
patients["FECHA_INGRESO"] = patients["FECHA_EGRESO"] - (patients["DIAS_ESTADA"] - timedelta(days=1))

In [16]:
patients.head()

,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,SEXO,EDAD_CANT,TIPO_EDAD,ANO_EGRESO,FECHA_EGRESO,AREA_FUNCIONAL_EGRESO,DIAS_ESTADA,CONDICION_EGRESO,DIAG1,GLOSA_DIAG1,DIAG2,GLOSA_DIAG2,INTERV_Q,PROCED,CODIGO_PROCED_PPAL,FECHA_INGRESO
0,110150,Hospital San José (Melipilla),1,33,1,2020,2020-08-27,403,6 days,1,N481,BALANOPOSTITIS,NaN,NaN,2,2,NaN,2020-08-22
1,110150,Hospital San José (Melipilla),2,81,1,2020,2020-01-22,403,11 days,1,I470,ARRITMIA POR REENTRADA VENTRICULAR,NaN,NaN,2,2,NaN,2020-01-12
2,110150,Hospital San José (Melipilla),1,3,3,2020,2020-09-16,413,1 days,1,P599,"ICTERICIA NEONATAL, NO ESPECIFICADA",NaN,NaN,2,2,NaN,2020-09-16
3,110150,Hospital San José (Melipilla),2,23,1,2020,2020-10-05,416,1 days,1,O600,TRABAJO DE PARTO PREMATURO SIN PARTO,NaN,NaN,2,2,NaN,2020-10-05
4,110150,Hospital San José (Melipilla),2,55,1,2020,2020-04-17,403,12 days,1,I500,INSUFICIENCIA CARDIACA CONGESTIVA,NaN,NaN,2,2,NaN,2020-04-06


## Sample for severity

In [17]:
patient_sample = patients.sample(n=30, random_state=70)
patient_sample.to_csv("data/severity_sample.csv")

## Simulate admissions and discharges from the hospital

In [18]:
# print(patients.groupby("DIAS_ESTADA").count())
frames = []
for i, patient in patients.iterrows():
    frames.append((patient["FECHA_INGRESO"], patient["FECHA_EGRESO"]))


In [19]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [20]:
n_in_hospital = []
n_admissions = []
n_discharges = []
n_same_day = []

start_date = date(2020, 1, 1)
end_date = date(2021, 1, 1)

for today in daterange(start_date, end_date):
    in_hospital = [(admission, discharge) for admission, discharge in frames if admission <= today <= discharge]
    same_day = [(admission, discharge) for admission, discharge in in_hospital if admission == discharge]
    n_same_day.append(len(same_day))
    n_in_hospital.append(len(in_hospital))
    admissions = [frame for frame in frames if frame[1].date() == today]
    n_admissions.append(len(admissions))
    discharges = [frame for frame in frames if frame[1].date() == today]
    n_discharges.append(len(discharges))

    # print(f"El {today}, hay {len(in_hospital)} pacientes y {len(discharges)} egresaron.")

print(f"Promedio de personas al día: {round(sum(n_in_hospital)/365, 1)}")
print(f"El máximo de pacientes fue {max(n_in_hospital)} y el mínimo {min(n_in_hospital)}")
print(f"Promedio de admisiones al dia {round(sum(n_admissions)/365, 1)}")
print(f"Promedio de egresos al dia {round(sum(n_discharges)/365, 1)}")
print(f"Tasa de egreso {round(sum(n_discharges) / sum(n_in_hospital) * 100, 1)}%")
print(f"Tasa un-día {round(sum(n_same_day) / sum(n_in_hospital) * 100, 1)}%")


Promedio de personas al día: 94.1
El máximo de pacientes fue 122 y el mínimo 27
Promedio de admisiones al dia 16.2
Promedio de egresos al dia 16.2
Tasa de egreso 17.2%
Tasa un-día 3.0%
